[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Amyssjj/Agent_Influcencer/blob/main/Agent_Youtube_video_insights.ipynb)

### **Motivation**
<a name="Motivation"></a>

##### In such a fast-paced world where information on the internet is exploding, staying connecte, forming my own perspective, and staying sharp is incredibly challenging. Yet, with full passion for learning and growth, I’m always striving to improve.


<hr>        






### **Pre-requisition**

In [3]:
%%capture
!pip install youtube_transcript_api elevenlabs langfun --pre

In [4]:
# Let's add path of virtual environment site-packages to system path
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/jing_env_colab/colab_env/lib/python3.10/site-packages")
sys.path.append('/content/drive/MyDrive/ColabNotebooks')  # Replace with the folder path


import langfun as lf
import pyglove as pg
from typing import Literal
import pandas as pd
import numpy as np
import os
import google.generativeai as genai
from google.colab import userdata
from langfun.core.structured import function_generation

Mounted at /content/drive


### **Build your crew to help you achieve your goal,  choose them wisely!**
<a name="Build your crew"></a>

In [5]:
## Get the key and LLMs
gemini_key = userdata.get('gemini_key')
claude_key = userdata.get('claude_key')
openai_key = userdata.get('openai_key')
youtube_key = userdata.get('youtube_key')

## This is your crew, help to achieve your goal/tasks!! Choose them wisely, since each of them have different expertise.
lm_claude = lf.llms.Claude35Sonnet20241022(api_key=claude_key, temperature=0.6) ## good at coding
lm_openai = lf.llms.Gpt4o(api_key = openai_key, temperature=0.6) ## good at summarization

### **Task 1 - Help you Write the code to retrieval all the information from a Youtube Video**


In [6]:
code = lf.query(prompt = "Help me to write a python code that achieve retreival the youtube video basic info, caption, and comments",
                # ==========================================================
                # Prompt SECTION: It's your request,  what you want LLM to do?
                # ==========================================================

                lm = lm_claude,
                # ==========================================================
                # lm SECTION: Your LLM Crew:  eg, Use Claude to write code
                # ==========================================================

                schema=lf.PythonCode,
                # ==========================================================
                # schema SECTION: Your desired output, eg, this output should Python code
                # ==========================================================
    )

In [8]:
%%capture
code

In [9]:
## Simply copy paste the code above,  and replace the API_KEY and Youtube_id you need to fetch!

from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi

# Replace with your API key
API_KEY = youtube_key

def get_video_info(video_id):
    # Create YouTube API client
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    # Get video details
    video_response = youtube.videos().list(
        part='snippet,statistics',
        id=video_id
    ).execute()

    if not video_response['items']:
        return None

    video_info = video_response['items'][0]

    # Get video captions
    try:
        captions = YouTubeTranscriptApi.get_transcript(video_id)
    except:
        captions = None

    # Get video comments
    comments = []
    try:
        comments_response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100
        ).execute()

        for item in comments_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'author': comment['authorDisplayName'],
                'text': comment['textDisplay'],
                'likes': comment['likeCount'],
                'published_at': comment['publishedAt']
            })
    except:
        pass

    return {
        'title': video_info['snippet']['title'],
        'description': video_info['snippet']['description'],
        'view_count': video_info['statistics']['viewCount'],
        'like_count': video_info['statistics']['likeCount'],
        'comment_count': video_info['statistics']['commentCount'],
        'captions': captions,
        'comments': comments
    }



In [10]:
# Example usage:
video_id = 'VMj-3S1tku0'  # Replace with actual video ID
video_data = get_video_info(video_id)

In [11]:
video_data['captions'][2]

{'text': 'networks for a bit more than a decade',
 'start': 2.96,
 'duration': 3.839}

In [12]:
video_data['comments'][10]

{'author': '@shreyaagarwal1123',
 'text': 'really thank u for making me understand  how neural network actually works makes me think how useless my uni actually is',
 'likes': 0,
 'published_at': '2024-11-24T10:35:48Z'}

### **Task 2 - Help you learn the key insights from video and comments,  and start your learning/insights/understanding**

In [13]:
class MajorInsights(pg.Object):  ## Your output, you define it
  summary: str
  key_insights: str
  questions: str

major_insights = lf.query(prompt = "{{video_data}}", video_data=video_data, schema=MajorInsights, lm=lm_openai)

In [14]:
major_insights

MajorInsights(summary='The video provides an in-depth tutorial on building a simple neural network from scratch using Python. It focuses on the theory and implementation of backpropagation, a fundamental algorithm in neural networks, using a minimalistic library called micrograd. The tutorial walks through the creation of a neural network, explaining each step in detail, and compares the implementation to modern deep learning frameworks like PyTorch.', key_insights='1. Backpropagation is the core algorithm for training neural networks, allowing for efficient computation of gradients.\n2. Neural networks can be broken down into simple mathematical expressions, and understanding these expressions is crucial for understanding how neural networks function.\n3. Implementing a neural network from scratch provides valuable insights into the underlying mechanics of deep learning frameworks.\n4. The tutorial emphasizes the importance of understanding derivatives and the chain rule in calculus for backpropagation.\n5. The video highlights common pitfalls in neural network implementation, such as the need to reset gradients before each backward pass.', questions='1. How does the choice of activation function, such as tanh or ReLU, impact the performance and training of a neural network?\n2. What are the advantages and disadvantages of implementing a neural network from scratch compared to using a library like PyTorch?\n3. How does backpropagation handle the vanishing gradient problem, and what are some strategies to mitigate it?\n4. How can we effectively choose learning rates and other hyperparameters to optimize neural network training?')